In [1]:
""" This will create a local cached version of the requests that I can use for testing.."""
import requests
import requests_cache
##jeckhart/requests-cache   |   0.4.10 | conda           | osx-64
#conda install -c https://conda.anaconda.org/USERNAME PACKAGE

import pymongo
#requests_cache.install_cache('isic_cache')  ##Creates local sqllite database


girder_token = None
def login(username, password):
    login_url = 'https://isic-archive.com/api/v1/user/authentication'
    login_response = requests.get(login_url, auth=(username, password))
    login_response.raise_for_status()
    global girder_token
    girder_token = login_response.json()['authToken']['token']

def apiGet(endpoint, as_json=True):
    url = 'https://isic-archive.com/api/v1/%s' % endpoint
    headers = {}
    global girder_token
    if girder_token:
        headers['Girder-Token'] = girder_token
    response = requests.get(url, headers=headers)
    if as_json:
        return response.json()
    else:
        return response

def getStudies():
    """ Get a list of all studies. """
    return apiGet('study')

def getStudyDetail(study_id):
    """ Get the detailed metadata for a study. """
    return apiGet('study/%s' % study_id)

def getFeaturesetDetail(featureset_id):
    """ Get the details of a featureset. """
    return apiGet('featureset/%s' % featureset_id)

def getStudyUsers(study_id):
    """ Get a list of users in a study. """
    return apiGet('study/%s/users' % study_id)

def getStudyImages(study_id):
    """ Get a list of images in a study. """
    return apiGet('study/%s/images' % study_id)

def getImageDetail(image_id):
    """ Get the detailed metadata for an image. """
    return apiGet('image/%s' % image_id)

def getImageFile(image_id):
    """ Get the original JPEG file for an image. """
    return apiGet('image/%s/download' % image_id, as_json=False)

def getImageThumbnail(image_id, width=None):
    """ Get the thumbnail-size JPEG preview for an image.
        Width defaults to 256 if not specified. """
    url = 'image/%s/thumbnail' % image_id
    if width is not None:
        url += '?width=%s' % width
    return apiGet(url, as_json=False)

def getAnnotations(study_id, user_id=None, image_id=None):
    """ Get the list of annotations for a study (optionally filtering by user or image). """
    url = 'annotation?studyId=' + study_id
    if user_id:
        url += '&userId=' + user_id
    if image_id:
        url += '&imageId=' + image_id
    return apiGet(url)

def getAnnotationDetail(annotation_id):
    """ Get the details of an annotation, including the feature values. """
    return apiGet('annotation/%s' % annotation_id)

def getSegmentationDetail(segmentation_id):
    """ Get the details of a segmentation, including the lesion boundary. """
    return apiGet('segmentation/%s' % segmentation_id)

def getSegmentationSuperpixels(segmentation_id):
    """ Get the PNG-encoded superpixels file for a segmentation. """
    return apiGet('segmentation/%s/superpixels' % segmentation_id, as_json=False)

def getSegmentationThumbnail(segmentation_id, width=None):
    """ Get the a thumbnail-size JPEG preview of the lesion boundary for a segmentation.
    Width defaults to 256 if not specified. """
    url = 'segmentation/%s/thumbnail' % segmentation_id
    if width is not None:
        url += '?width=%s' % width
    return apiGet(url, as_json=False)

In [2]:
""" Enter your credentials here, but DO NOT save and commit this Notebook with them present. """
from secrets import username,password


login(username, password)

In [3]:
study_list = getStudies()
for study in study_list:
    study_featureset = getFeaturesetDetail(study['meta']['featuresetId'])
    study_images = getStudyImages(study['_id'])
    study_users = getStudyUsers(study['_id'])
    
    print 'Study: %s (%s)' % (study['name'], study['_id'])
    print '  Featureset: %s' % study_featureset['name']
    print '  Images: %s' % len(study_images)
    print '  Users: %s' % len(study_users)

Study: MSK-2 (55c405419fc3c151ac90bfb6)
  Featureset: ISIC Conventional
  Images: 1535
  Users: 4
Study: sonic_br_bris_bros (55c442879fc3c1536a012d60)
  Featureset: UDA2 Study
  Images: 400
  Users: 1
Study: uda_10-9-2014_560imgs (55c442809fc3c1536a012b57)
  Featureset: UDA2 Study
  Images: 259
  Users: 1
Study: sonic_exemplars_for_noel (55c4429b9fc3c1536a012ef2)
  Featureset: UDA2 Study
  Images: 100
  Users: 1
Study: SONIC_2_2_2015 (55c4427e9fc3c1536a012a8c)
  Featureset: UDA2 Study
  Images: 100
  Users: 1
Study: UDA2_pilot (55c4668a9fc3c1536a0130c8)
  Featureset: UDA2 Study
  Images: 60
  Users: 4
Study: ISBI Challenge (567864779fc3c148a0c9b248)
  Featureset: ISBI Challenge
  Images: 664
  Users: 3
Study: ISBI Challenge (MSK-2) (568c0a4c9fc3c133e7ddc20a)
  Featureset: ISBI Challenge
  Images: 615
  Users: 1


In [4]:
study = getStudyDetail('55c4668a9fc3c1536a0130c8')  # UDA2_pilot
study_images = getStudyImages(study['_id'])
study_users = getStudyUsers(study['_id'])

print study['name']
for image in study_images:
    print ' ', image['name']
    # Fetch the details, so we can get metadata
    image = getImageDetail(image['_id'])
    for user in study_users:
        print '   ', user['login'],
        annotations = getAnnotations(
            study_id=study['_id'],
            image_id=image['_id'],
            user_id=user['_id']
        )
        # A study + image + user should fully define a single annotation
        assert len(annotations) == 1
        
        # We need to individually fetch annotations in order to get the feature values
        annotation = getAnnotationDetail(annotations[0]['_id'])
        
        if annotation.get('stopTime'):
            print 'complete'
        else:
            print 'pending'
            continue
        
        if annotation.get('segmentationId'):
            segmentation = getSegmentationDetail(annotation['segmentationId'])
        else:
            # Some old annotations don't have the segmentation ID attached yet
            # So, grab the first (most recent) segmentation associated with the image
            segmentation = apiGet('segmentation?imageId=%s' % annotation['imageId'])[0]

        image_metadata = image['meta']
        superpixels = getSegmentationSuperpixels(segmentation['_id'])
        feature_values = annotation['annotations']
        # There is some inconsistancy in how the annotations are currently stored
        if 'region_features' in feature_values:
            feature_values = feature_values['region_features']

        # TODO: do something with the image_metadata, superpixels, and feature_values


UDA2_pilot
  ISIC_0001100
    braunr complete
    carrerac complete
    haroldr complete
    marghooa complete
  ISIC_0001101
    braunr complete
    carrerac complete
    haroldr complete
    marghooa complete
  ISIC_0001102
    braunr complete


KeyboardInterrupt: 

In [5]:
## So now what we really want to play around with, is for a given image (which we look up by ID)... lets get the samn results!
sample_studyId = '55c4668a9fc3c1536a0130c8'

study = getStudyDetail(sample_studyId)  # UDA2_pilot
study_images = getStudyImages(study['_id'])
study_users = getStudyUsers(study['_id'])

def _makeUrl(endpoint):
    API_URL = 'https://isic-archive.com/api/v1'
    return '%s/%s' % (API_URL, endpoint)


In [8]:
from IPython.display import display,Image
import IPython
sample_image =  study_images[10]
print sample_image
##  Now for this image and this study, who drew stuff on it... I also acutally want to look at the image..
## Frst let's display image
getImageThumbnail(sample_image['_id'])
thumb_url = _makeUrl('image/%s/thumbnail?width=300' % sample_image['_id'])
display(Image(url=thumb_url))

## Now I am curious to see what "stuff" was marked up..


{u'_id': u'54e755f7bae47850e86cdff9', u'name': u'ISIC_0001110'}


In [14]:
## Now let's get some data?? i.e. markups
## First thing is to get the annotations associated with a specific image... this is really just the uniqueID keys, not the data
image_annotation_set = getAnnotations(sample_studyId, image_id=sample_image['_id'])
print study_users

studyUsers_ToEnglish = {}
for x in study_users:
    studyUsers_ToEnglish[x['_id']] = x['login']
    
print studyUsers_ToEnglish

 [{u'login': u'braunr', u'_id': u'54f9c702bae47807dbb0d5ba'}, {u'login': u'carrerac', u'_id': u'54f86f12bae47871b5e0506a'}, {u'login': u'haroldr', u'_id': u'54f9f7a3bae47807dbb0d836'}, {u'login': u'marghooa', u'_id': u'54cbe807bae47819d8e4c79a'}]
{u'54f9f7a3bae47807dbb0d836': u'haroldr', u'54f86f12bae47871b5e0506a': u'carrerac', u'54f9c702bae47807dbb0d5ba': u'braunr', u'54cbe807bae47819d8e4c79a': u'marghooa'}


In [28]:
### Lets get the actual annotations.. i.e. the ANSWERS people used..
#print image_annotation_set

#sys.exit()

user_annotations = []
for ias in image_annotation_set:
    #print ias,ias['userId'], studyUsers_ToEnglish[ias['userId']]
    ## Let's pull the annotation data for
    print "Pulling annotation data for",studyUsers_ToEnglish[ias['userId']]
    ua = getAnnotationDetail(ias['_id'])
    user_annotations.append(ua)

Pulling annotation data for marghooa
Pulling annotation data for braunr
Pulling annotation data for haroldr
Pulling annotation data for carrerac


In [51]:
#print user_annotation## OK MOHAMMED, what is in a user annotation. this is complicated, so here we go
import datetime
## At least for this study, the way we have things organized:
## studyID:  Self Explanatory
## imageId:  The same (probably want to keep the derefernced name handy in a helper dictionary)
## startTime and endTime;  We will want to do some statistics on these...
## Now it gets more complicated, what's in an annotation (and remeber this is what I really want to filter on..)
## annotation  -->  Two properties:  
#              image_features:  Apply to an ENTIRE image, like Tan Skin, Melanotic Lesion, etc, etc
#`             region_features:  These specify information about a given feature and what superpixels that was detected in
#               HOWEVER:  If a feature was not marked up, we didn't bother creating an "empty" array of all 0's
#               Another thing that's weird:  The array is indexed based on a superpixelID, which changes for each and every image


print user_annotations[2]['annotations'].keys()


for ua in user_annotations:
    #imageMarkupTime = date(ua['stopTime']) - date(ua['startTime'])
    #print ua['startTime'],ua['stopTime'],
    print ua['name'],ua['user']['login'],'marked up this image.. lets see what he thought...'
    ## #Now let's get the actual ANNOTATIONS... which are at the image and region level
    image_annots = ua['annotations']['image_features']
    region_annots = ua['annotations']['region_features']
    print "----IMAGE LEVEL FEATURES"
    for k,v in image_annots.iteritems():
        print k,v
    ## We cna't simply print the next set out.. too many damn numbes..
    ## LEt's see how many features overall this person may have evaluated...
    potential_markedup_features = region_annots.keys() ## In theory these features were markedup, although there's a weird exception
    ## So annoyingly, just beacause there's an array for a feature, it still potentially can contain all 0's.. c'est lavie
    for k,v in region_annots.iteritems():
        print len(v),'super pixels in this set and',sum(v),sum([1 for x in v if x>0])


        
    

[u'image_features', u'region_features']
ISIC_0001110 marghooa marked up this image.. lets see what he thought...
----IMAGE LEVEL FEATURES
organization org_no
patternsym symp_none
shapesym sym_oneaxis
pattern other
235 super pixels in this set and 29.0 29
235 super pixels in this set and 5.0 5
235 super pixels in this set and 8.0 8
235 super pixels in this set and 15.0 15
235 super pixels in this set and 6.0 6
235 super pixels in this set and 0.0 0
235 super pixels in this set and 3.0 3
235 super pixels in this set and 1.0 1
235 super pixels in this set and 4.0 4
ISIC_0001110 braunr marked up this image.. lets see what he thought...
----IMAGE LEVEL FEATURES
organization org_no
patternsym symp_none
shapesym sym_none
pattern other
235 super pixels in this set and 60.0 60
235 super pixels in this set and 2.0 2
235 super pixels in this set and 3.0 3
235 super pixels in this set and 6.0 6
235 super pixels in this set and 20.0 20
235 super pixels in this set and 0.0 0
235 super pixels in this